In [9]:
import json

with open('..\credentials.json') as f:
    keys = json.load(f)

In [11]:
import pandas as pd
import tweepy

# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

# Hacer un tuit

# api.update_status('tweet test')

In [12]:
tweets = tweepy.Cursor(api.search,
                       q=['giro de italia -RT'],
                       lang='es',
                       geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [13]:
# Objeto generador
t = [tweet for tweet in tweets]
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    t_processed.append(temp)
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,¡Mikel Landa deberá ser operado tras su accide...,2021-05-13 23:58:34,Bogotá,Bogotá,1392992698085388291,https://twitter.com/i/web/status/1392992698085...
1,¡Golpea el Ineos! 🇨🇴🔥🚴‍♂️Egan Bernal otra vez ...,2021-05-13 23:00:00,Bogotá,Bogotá. Colombia,1392977960177569793,https://twitter.com/i/web/status/1392977960177...
2,#DeportesRCN |VIDEO| 🇨🇴🚴‍♂️\n\nEgan Bernal hab...,2021-05-13 22:45:26,Bogotá,"Bogotá, Colombia",1392974292242112518,https://twitter.com/i/web/status/1392974292242...
3,Felicitaciones para el Ciclista colombiano @Eg...,2021-05-13 22:19:56,Bogotá,"Bogotá, D.C., Colombia",1392967876970229767,https://twitter.com/i/web/status/1392967876970...
4,"El ciclista colombiano Egan Bernal, se lució e...",2021-05-13 21:53:27,Bogotá,"Bogotá, D.C., Colombia",1392961213424979970,https://twitter.com/i/web/status/1392961213424...


In [15]:
data.to_csv("archivo.csv", index=False)

In [17]:
#datos historicos
import snscrape.modules.twitter as sntwitter

maxTweets = 10  
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#eragoldeyepes ' + 'since:2014-07-15 until:2014-08-15').get_items()) :
        if i > maxTweets :
            break
        print(tweet)
        print(tweet.date)
        print(tweet.content)
        print("*"*100)
        print()

https://twitter.com/CANmialegria/status/500066563522056192
2014-08-14 23:48:54+00:00
Cambiemos el HT #EraGolDeYepes, por #EraGolDeCopete. ¡Nunca hubo fuera de lugar!
¿#LaCulpaEsDePostobon?  @PradoVerde1999 @vamosmiverdecom
****************************************************************************************************

https://twitter.com/cami_canoles/status/500064190288384000
2014-08-14 23:39:28+00:00
http://t.co/X3CpWL1Q3x #eragoldeyepes
****************************************************************************************************

https://twitter.com/TamayoJr11/status/500062617969643520
2014-08-14 23:33:13+00:00
#EraGolDeYepes http://t.co/K9Am0bxOIh
****************************************************************************************************

https://twitter.com/Edw_Chacon/status/500055868361412608
2014-08-14 23:06:24+00:00
JjaJja Todavia Me Acuerdo :') #EraGolDeYepes http://t.co/xFkiku07ae
****************************************************************************

In [18]:
data = data[['tweets', 'date']]
data.head()

,tweets,date
0,¡Mikel Landa deberá ser operado tras su accide...,2021-05-13 23:58:34
1,¡Golpea el Ineos! 🇨🇴🔥🚴‍♂️Egan Bernal otra vez ...,2021-05-13 23:00:00
2,#DeportesRCN |VIDEO| 🇨🇴🚴‍♂️\n\nEgan Bernal hab...,2021-05-13 22:45:26
3,Felicitaciones para el Ciclista colombiano @Eg...,2021-05-13 22:19:56
4,"El ciclista colombiano Egan Bernal, se lució e...",2021-05-13 21:53:27


In [19]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,texto_limpio,links,hashtags,mentions
0,¡Mikel Landa deberá ser operado tras su accide...,2021-05-13 23:58:34,¡Mikel Landa deberá ser operado tras su accide...,[https://t.co/ekMmBdYX7J],[],[]
1,¡Golpea el Ineos! 🇨🇴🔥🚴‍♂️Egan Bernal otra vez ...,2021-05-13 23:00:00,¡Golpea el Ineos! 🇨🇴🔥🚴‍♂️Egan Bernal otra vez ...,"[https://t.co/6yQSA9HD7k, https://t.co/xuZF6er...",[],[]
2,#DeportesRCN |VIDEO| 🇨🇴🚴‍♂️\n\nEgan Bernal hab...,2021-05-13 22:45:26,|VIDEO| 🇨🇴🚴‍♂️\n\nEgan Bernal habló de la sex...,[https://t.co/n5CPSVWgUp],"[#DeportesRCN, #NuestroCiclismo]",[]
3,Felicitaciones para el Ciclista colombiano @Eg...,2021-05-13 22:19:56,Felicitaciones para el Ciclista colombiano qu...,[https://t.co/n8PTrNkyzB],[],[@Eganbernal]
4,"El ciclista colombiano Egan Bernal, se lució e...",2021-05-13 21:53:27,"El ciclista colombiano Egan Bernal, se lució e...",[https://t.co/G297IC28o5],[],[]
...,...,...,...,...,...,...
95,Gino Mader ganó la etapa 6 del Giro de Italia....,2021-05-13 15:17:42,Gino Mader ganó la etapa 6 del Giro de Italia....,[],[],[]
96,#Giro | Egan Bernal llegó segundo en la meta y...,2021-05-13 15:16:41,| Egan Bernal llegó segundo en la meta y el t...,[https://t.co/qCeEtMh0k9],[#Giro],[]
97,#Giro | Egan Bernal llegó segundo en la meta y...,2021-05-13 15:16:35,| Egan Bernal llegó segundo en la meta y el t...,[https://t.co/3qnAkaUN1v],[#Giro],[]
98,#Giro | Ganó Gino Mader la etapa 5 del Giro de...,2021-05-13 15:16:33,| Ganó Gino Mader la etapa 5 del Giro de Ital...,[https://t.co/cvmYhDb9W6],"[#Giro, #EnVivo]",[]


In [21]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
  

data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,texto_limpio,links,hashtags,mentions,emojis
0,¡Mikel Landa deberá ser operado tras su accide...,2021-05-13 23:58:34,¡Mikel Landa deberá ser operado tras su accide...,[https://t.co/ekMmBdYX7J],[],[],[]
1,¡Golpea el Ineos! 🇨🇴🔥🚴‍♂️Egan Bernal otra vez ...,2021-05-13 23:00:00,¡Golpea el Ineos! 🇨🇴🔥🚴‍♂️Egan Bernal otra vez ...,"[https://t.co/6yQSA9HD7k, https://t.co/xuZF6er...",[],[],"[🔥, 🚴‍♂️, ➡️, ➡️]"
2,#DeportesRCN |VIDEO| 🇨🇴🚴‍♂️\n\nEgan Bernal hab...,2021-05-13 22:45:26,|VIDEO| 🇨🇴🚴‍♂️\n\nEgan Bernal habló de la sex...,[https://t.co/n5CPSVWgUp],"[#DeportesRCN, #NuestroCiclismo]",[],[🚴‍♂️]
3,Felicitaciones para el Ciclista colombiano @Eg...,2021-05-13 22:19:56,Felicitaciones para el Ciclista colombiano qu...,[https://t.co/n8PTrNkyzB],[],[@Eganbernal],[]
4,"El ciclista colombiano Egan Bernal, se lució e...",2021-05-13 21:53:27,"El ciclista colombiano Egan Bernal, se lució e...",[https://t.co/G297IC28o5],[],[],[]


In [22]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
0,15:16:00,4
8,15:17:00,2
6,15:18:00,2
62,15:21:00,1
22,15:22:00,1
...,...,...
59,21:53:00,1
18,22:19:00,1
63,22:45:00,1
67,23:00:00,1


In [23]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)


trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [26]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
1,🚴‍♂️,23
4,💗,11
2,➡️,7
6,👏,6
5,🏆,5


In [27]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)